In [35]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
# from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models, utils
from tqdm import tqdm
import numpy as np
import pandas as pd
# from torchsummary import summary


In [33]:

class ConvNet(nn.Module):

    def __init__(self):
        super().__init__()
        #self.conv1 = nn.Conv2d(1, 16, 11, padding=5)
        #self.conv2 = nn.Conv2d(16, 16, 7, padding=3)
        #self.conv3 = nn.Conv2d(16, 16, 5, padding=2)
        self.conv3 = nn.Conv2d(1, 16, 5, padding=2)
        #self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1)
        #self.conv5 = nn.Conv2d(16, 1, 3, padding=1)
        self.conv6 = nn.Conv2d(16, 1, 1, padding=0)
        self.fc1 = nn.Linear(28 * 28, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        
        #x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        #x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [36]:

class SparseDatasetMNIST(Dataset):
    def __init__(self, file_path, transform=None, sparsity_level=0.0):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        self.sparsity_level = sparsity_level

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # generate the random mask
        tensor_length = 28 * 28
        ones_count = int(tensor_length * self.sparsity_level)
        ones_indices = torch.randperm(tensor_length)[:ones_count]
        random_mask = torch.zeros(tensor_length).bool()
        random_mask[ones_indices] = 1
        random_mask = random_mask.reshape((28, 28, 1))

        # generate the sparse image
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28, 1))
        image = torch.from_numpy(image)
        processed_image = image.masked_fill(random_mask, 0).numpy()
        label = self.data.iloc[index, 0]

        if self.transform is not None:
            # print("---")
            processed_image = self.transform(processed_image)

        
        #return processed_image, random_mask.reshape((1,28,28)).float(), label
        return processed_image, label

In [37]:
def train(train_loader, model, optimizer, criterion, device):
    """
    Trains network for one epoch in batches.

    Args:
        train_loader: Data loader for training set.
        model: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        criterion: Loss function (e.g. cross-entropy loss).
        device: whether the network runs on cpu or gpu
    """

    avg_loss = 0
    correct = 0
    total = 0

    # Switch to train mode
    model.train()

    # Iterate through batches
    for i, data in enumerate(train_loader):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # ##########
        # # Get the inputs; data is a list of [inputs, labels]
        # inputs, mask, labels = data

        # # Move data to target device
        # inputs, mask, labels = inputs.to(device), mask.to(device), labels.to(device)

        # # Zero the parameter gradients
        # optimizer.zero_grad()

        # # Forward + backward + optimize
        # # Forward propagation
        # mask = (inputs > 0).float()
        # outputs = model(inputs, mask)

        # # Loss computation
        # loss = (criterion(outputs, labels) * mask.detach()).sum() / mask.sum()
        # ############

        loss.backward()
        optimizer.step()

        # Keep track of loss and accuracy
        avg_loss += loss
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return avg_loss / len(train_loader), 100 * correct / total

In [38]:
def test(test_loader, model, criterion, device):
    """
    Evaluates network in batches.

    Args:
        test_loader: Data loader for test set.
        model: Neural network model.
        criterion: Loss function (e.g. cross-entropy loss).
    """

    avg_loss = 0
    correct = 0
    total = 0

    # Switch to test mode
    model.eval()

    # Use torch.no_grad to skip gradient calculation, not needed for evaluation
    with torch.no_grad():
        # Iterate through batches
        for data in test_loader:
            
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            

            # ###########
            # # Get the inputs; data is a list of [inputs, labels]
            # inputs, mask, labels = data

            # # Move data to target device
            # inputs, mask, labels = inputs.to(device), mask.to(device), labels.to(device)

            # # Forward + backward + optimize
            # # Forward propagation
            # mask = (inputs > 0).float()
            # outputs = model(inputs, mask)

            # # Loss computation
            # loss = (criterion(outputs, labels) * mask.detach()).sum() / mask.sum()
            # ###########


            # Keep track of loss and accuracy
            avg_loss += loss
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return avg_loss / len(test_loader), 100 * correct / total

In [25]:
# hyperparameters
n_epochs = 10
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
weight_decay = 1e-3
#momentum = 0.5
log_interval = 10
random_seed = 42
torch.manual_seed(random_seed)
spar_level = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# mps_avail = torch.backends.mps.is_available()
# print(f"Is the mps available? {mps_avail}")


In [31]:

the_transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,)),
                       ])
spar_level = 0.3
training_set = SparseDatasetMNIST(file_path='C:/Users/79003/Desktop/mnist/data/mnist_train.csv', transform=the_transform, sparsity_level=spar_level)
train_loader = DataLoader(training_set, batch_size=batch_size_train, shuffle=True)

test_set = SparseDatasetMNIST(file_path='C:/Users/79003/Desktop/mnist/data/mnist_test.csv', transform=the_transform, sparsity_level=spar_level)
test_loader = DataLoader(test_set, batch_size=batch_size_test, shuffle= False)

print(len(training_set))
print(len(test_set))
print(type(train_loader))
print(type(test_loader))
# # draw some examples
# examples = enumerate(test_loader)
# #batch_idx, (example_data, example_mask, example_targets) = next(examples)
# batch_idx, (example_data, example_targets) = next(examples)
# # print(example_targets)
# # print(example_data.shape)
# '''
# fig = plt.figure()
# for i in range(6):
#     plt.subplot(2, 3, i + 1)
#     plt.tight_layout()
#     plt.imshow(example_mask[i][0], cmap='gray', interpolation='none')
#     plt.title("Example random mask")
#     plt.xticks([])
#     plt.yticks([])
# plt.show()
# '''
# fig = plt.figure()
# for i in range(6):
#     plt.subplot(2, 3, i + 1)
#     plt.tight_layout()
#     plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
#     plt.title("Ground Truth: {}".format(example_targets[i]))
#     plt.xticks([])
#     plt.yticks([])
# plt.show()



60000
10000
<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader.DataLoader'>


In [32]:

spar_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
batch_size_train = 64
batch_size_test = 1000
n_epochs = 10  
learning_rate = 0.005
weight_decay = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in range(len(spar_levels)):
    the_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    sparsity_level = spar_levels[i]

    training_set = SparseDatasetMNIST(file_path='C:/Users/79003/Desktop/mnist/data/mnist_train.csv',
                                      transform=the_transform, sparsity_level=sparsity_level)
    train_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size_train, shuffle=True)

    test_set = SparseDatasetMNIST(file_path='C:/Users/79003/Desktop/mnist/data/mnist_test.csv',
                                  transform=the_transform, sparsity_level=sparsity_level)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test, shuffle=False)

    model = ConvNet()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    model = model.to(device)

    train_losses = []
    train_accuracy = []
    train_counter = []
    test_losses = []
    test_accuracy = []

    for epoch in tqdm(range(n_epochs)):
        # Train on data
        train_loss, train_acc = train(train_loader, model, optimizer, criterion, device)
        train_losses.append(train_loss)
        train_accuracy.append(train_acc)
        print('\nTraining for Sparsity Level: {}'.format(sparsity_level))
        print('\nTraining Loss: {:.4f}, Training Acc: {:.4f}'.format(train_loss, train_acc))

        # Test on data
        test_loss, test_acc = test(test_loader, model, criterion, device)
        test_losses.append(test_loss)
        test_accuracy.append(test_acc)

    print('\nFinished training for Sparsity Level: {}'.format(sparsity_level))
    print('\nTest Loss: {:.4f}, Test Acc: {:.4f}'.format(test_loss, test_acc))


  0%|          | 0/10 [00:00<?, ?it/s]


Training for Sparsity Level: 0

Training Loss: 0.2426, Training Acc: 92.3817


 10%|█         | 1/10 [00:14<02:07, 14.13s/it]


Training for Sparsity Level: 0

Training Loss: 0.1252, Training Acc: 96.1717


 20%|██        | 2/10 [00:28<01:52, 14.11s/it]

...


Training for Sparsity Level: 0.9

Training Loss: 0.7118, Training Acc: 75.7933


 90%|█████████ | 9/10 [02:12<00:14, 14.81s/it]


Training for Sparsity Level: 0.9

Training Loss: 0.7192, Training Acc: 75.7317


100%|██████████| 10/10 [02:27<00:00, 14.70s/it]


Finished training for Sparsity Level: 0.9

Test Loss: 0.7041, Test Acc: 75.5500


In [39]:

spar_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95]
batch_size_train = 64
batch_size_test = 1000
n_epochs = 20  
learning_rate = 0.005
weight_decay = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in range(len(spar_levels)):
    the_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    sparsity_level = spar_levels[i]

    training_set = SparseDatasetMNIST(file_path='C:/Users/79003/Desktop/mnist/data/mnist_train.csv',
                                      transform=the_transform, sparsity_level=sparsity_level)
    train_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size_train, shuffle=True)

    test_set = SparseDatasetMNIST(file_path='C:/Users/79003/Desktop/mnist/data/mnist_test.csv',
                                  transform=the_transform, sparsity_level=sparsity_level)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test, shuffle=False)

    model = ConvNet()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    model = model.to(device)

    train_losses = []
    train_accuracy = []
    train_counter = []
    test_losses = []
    test_accuracy = []

    for epoch in tqdm(range(n_epochs)):
        # Train on data
        train_loss, train_acc = train(train_loader, model, optimizer, criterion, device)
        train_losses.append(train_loss)
        train_accuracy.append(train_acc)
        print('\nTraining for Sparsity Level: {}'.format(sparsity_level))
        print('\nTraining Loss: {:.4f}, Training Acc: {:.4f}'.format(train_loss, train_acc))

        # Test on data
        test_loss, test_acc = test(test_loader, model, criterion, device)
        test_losses.append(test_loss)
        test_accuracy.append(test_acc)

    print('\nFinished training for Sparsity Level: {}'.format(sparsity_level))
    print('\nTest Loss: {:.4f}, Test Acc: {:.4f}'.format(test_loss, test_acc))


  0%|          | 0/20 [00:00<?, ?it/s]


Training for Sparsity Level: 0

Training Loss: 0.2427, Training Acc: 92.5750


  5%|▌         | 1/20 [00:14<04:44, 14.99s/it]


Training for Sparsity Level: 0

Training Loss: 0.1419, Training Acc: 95.6667


 10%|█         | 2/20 [00:28<04:24, 14.68s/it]

...


Training for Sparsity Level: 0.95

Training Loss: 1.3836, Training Acc: 52.3250


 95%|█████████▌| 19/20 [04:32<00:14, 14.38s/it]


Training for Sparsity Level: 0.95

Training Loss: 1.3305, Training Acc: 53.7050


100%|██████████| 20/20 [04:47<00:00, 14.36s/it]


Finished training for Sparsity Level: 0.95

Test Loss: 1.2574, Test Acc: 56.2200
